In [7]:
import numpy as np

# Choose the filename of the uploaded file
filename_true = "/content/drive/MyDrive/word_data6/true_input4.txt"

# Read the content of the file line by line and store it in a NumPy array
lines = []
with open(filename_true, 'r') as file:
    for line in file:
        lines.append(line.strip())

# Convert the list of lines to a NumPy array
np_true = np.array(lines)

filename_true_labels= "/content/drive/MyDrive/word_data6/output_label4.txt"
lines = []
with open(filename_true_labels, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_true_label= np.array(lines)


filename_false= "/content/drive/MyDrive/word_data6/false_data4.txt"
lines = []
with open(filename_false, 'r') as file:
    for line in file:
        lines.append(line.strip())
np_false= np.array(lines)


filename_false_labels= "/content/drive/MyDrive/word_data6/false_data_labels4.txt"
lines = []
with open(filename_false_labels, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_false_label= np.array(lines)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [8]:
filename_true2 = "/content/drive/MyDrive/word_data5/true_input3.txt"

# Read the content of the file line by line and store it in a NumPy array
lines = []
with open(filename_true2, 'r') as file:
    for line in file:
        lines.append(line.strip())

# Convert the list of lines to a NumPy array
np_true2 = np.array(lines)

filename_true_labels2= "/content/drive/MyDrive/word_data5/output_label3.txt"
lines = []
with open(filename_true_labels2, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_true_label2= np.array(lines)


filename_false2= "/content/drive/MyDrive/word_data5/false_data3.txt"
lines = []
with open(filename_false2, 'r') as file:
    for line in file:
        lines.append(line.strip())
np_false2= np.array(lines)


filename_false_labels2= "/content/drive/MyDrive/word_data5/false_data_labels3.txt"
lines = []
with open(filename_false_labels2, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_false_label2= np.array(lines)

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_total is your input data and y_total is your labels

subset_size = len(np_false) // 10
random_indices = np.random.choice(len(np_false), size=subset_size, replace=False)
subset_false = np_false[random_indices]

# Concatenate np_true with the randomly chosen subset of np_false
X_total = np.concatenate((np_true, subset_false), axis=0)
y_total = np.concatenate((np_true_label, np_false_label[random_indices]), axis=0)

# Randomly choose a subset of np_false2
subset_size2 = len(np_false2) // 10
random_indices2 = np.random.choice(len(np_false2), size=subset_size2, replace=False)
subset_false2 = np_false2[random_indices2]

# Concatenate np_true2 with the randomly chosen subset of np_false2
X_total2 = np.concatenate((np_true2, subset_false2), axis=0)
y_total2 = np.concatenate((np_true_label2, np_false_label2[random_indices2]), axis=0)
# Calculate the size of the reduced dataset (1/10)
fraction_to_keep = 0.95  # 10% of the data
fraction_to_keep2 = 0.95
# Use train_test_split to randomly select a subset
X_reduced, _, y_reduced, _ = train_test_split(X_total, y_total, test_size=1-fraction_to_keep, random_state=42)
X_reduced2, _, y_reduced2, _ = train_test_split(X_total2, y_total2, test_size=1-fraction_to_keep2, random_state=42)

y_reduced=y_reduced.astype(int)
y_reduced2=y_reduced2.astype(int)
# Split the reduced data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_reduced, y_reduced, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_reduced2, y_reduced2, test_size=0.2, random_state=42)

In [51]:
X_train= X_reduced2
y_train = y_reduced2

In [10]:
X_train = np.concatenate((X_train,X_train2), axis=0)
y_train = np.concatenate((y_train,y_train2), axis=0)

In [11]:
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [12]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])
X_test2 = np.array([vectorize(sentence) for sentence in X_test2])
#max_len = max(len(sentence.split()) for sentence in X_train)
#X_train = pad_sequences(X_train, maxlen=100, padding='post')
#X_test = pad_sequences(X_test, maxlen=100, padding='post')

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# Assuming `X_train` is your input data and `y_train` is your labels

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model Architecture
# Create a Sequential model
model = Sequential()

# Add an Embedding layer if your input is text data
# model.add(Embedding(vocabulary_size, embedding_dim, input_length=max_len))

# Add a Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(X_train.shape[1], 1)))  # Assuming X_train is a 2D array

# Add a Dense layer for classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test2, y_test2), callbacks=[early_stopping])

# Evaluate on the test set
y_test_pred = model.predict(X_test)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred_binary)
print(f"Test Accuracy: {accuracy}")
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_pred_binary)
recall = recall_score(y_test, y_test_pred_binary)
f1 = f1_score(y_test, y_test_pred_binary)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Epoch 1/5
2016/2016 [==============================] - 294s 144ms/step - loss: 76101.5312 - accuracy: 0.6258 - val_loss: 0.5745 - val_accuracy: 0.8033
Epoch 2/5
2016/2016 [==============================] - 281s 140ms/step - loss: 5.5005 - accuracy: 0.7156 - val_loss: 0.5740 - val_accuracy: 0.8031
Epoch 3/5
2016/2016 [==============================] - 297s 148ms/step - loss: 0.5923 - accuracy: 0.7398 - val_loss: 0.5496 - val_accuracy: 0.7962
Epoch 4/5
2016/2016 [==============================] - 284s 141ms/step - loss: 0.5674 - accuracy: 0.7479 - val_loss: 0.5300 - val_accuracy: 0.7924
Epoch 5/5
359/359 [==============================] - 14s 37ms/step
Test Accuracy: 0.700985952360178
Confusion Matrix:
[[5094  513]
 [2914 2940]]
Precision: 0.8514335360556038
Recall: 0.5022207037922788
F1 Score: 0.6317825292790373


In [77]:
strings_list = ["patlıcanları ince doğraki güzel pişsin", "kiminki kazanacak göreceğiz", "teyzeminki daha yeni duruyor",
                "şöyle gelki yüzünü göreyim", "dağın yamaçlarındaki karlar eridi"
                ,"ara sıra öyküler yazki yazın güçlensin","ilaçlarını içki çabuk iyileş","biliyorsunki bu film ödül aldı","Ahemti tembihleki ağızından birşey kaçırmasın",
                "Birşey biliyorki sesini çıkarmıyor","Bu yarışmayı kazanabilecek misinki",
                "eve döndüm baktımki gitmiş hemen telefona sarıldım","Senki benim en iyi arkadaşımdın",
                "gel görki ne bağ kırılmış ne bahçe","bir şey biliyorki konuşuyor.",
                "sen de onunla gitki, ona yardımcı olur","Kırıkları toplaki, elimize batmasın"
                ,"bizimle gelki biraz hava al","yağmur yağmadıki mantarlar ortaya çıksın.","tam evden çıkıyorduki telefon çaldı."]

# Create a NumPy array from the list of strings
test_labels=[0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
test_labels=np.array(test_labels)
my_sentences = np.array(strings_list)

In [78]:
test_my_sentences= np.array([vectorize(sentence) for sentence in my_sentences])
#test_my_sentences = test_my_sentences.reshape((test_my_sentences.shape[0], 1, test_my_sentences.shape[1]))
y_test_pred = model.predict(test_my_sentences)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)
accuracy=accuracy_score(test_labels,y_test_pred_binary)
print(f"Test Accuracy: {accuracy}")
# Calculate accuracy
for i in range(len(my_sentences)):
  print(f"Test:{my_sentences[i]} Prediction:{y_test_pred_binary[i]}")

1/1 [==============================] - 0s 71ms/step
Test Accuracy: 0.7
Test:patlıcanları ince doğraki güzel pişsin Prediction:[1]
Test:kiminki kazanacak göreceğiz Prediction:[1]
Test:teyzeminki daha yeni duruyor Prediction:[1]
Test:şöyle gelki yüzünü göreyim Prediction:[0]
Test:dağın yamaçlarındaki karlar eridi Prediction:[0]
Test:ara sıra öyküler yazki yazın güçlensin Prediction:[0]
Test:ilaçlarını içki çabuk iyileş Prediction:[0]
Test:biliyorsunki bu film ödül aldı Prediction:[0]
Test:Ahemti tembihleki ağızından birşey kaçırmasın Prediction:[0]
Test:Birşey biliyorki sesini çıkarmıyor Prediction:[0]
Test:Bu yarışmayı kazanabilecek misinki Prediction:[0]
Test:eve döndüm baktımki gitmiş hemen telefona sarıldım Prediction:[0]
Test:Senki benim en iyi arkadaşımdın Prediction:[0]
Test:gel görki ne bağ kırılmış ne bahçe Prediction:[1]
Test:bir şey biliyorki konuşuyor. Prediction:[1]
Test:sen de onunla gitki, ona yardımcı olur Prediction:[1]
Test:Kırıkları toplaki, elimize batmasın Prediction